In [1]:
import time
import N38_scraper
from N38_scraper import get_allnews_urls
from N38_scraper import parse_page

urls = get_allnews_urls(begin_page=1, end_page=4, verbose=True)
print('%d urls for all news' % len(urls))

get briefing statement urls 1 / 4
get briefing statement urls 2 / 4
get briefing statement urls 3 / 4
get briefing statement urls 4 / 4
16 urls for all news


In [2]:
def pprint(json_object):
    for k, v in json_object.items():
        print('{} : {} ..'.format(k, str(v)[:100]))
    print('\n')

SLEEP = 0.5

for url in urls[:10]:
    json_object = parse_page(url)
    pprint(json_object)

title : How Vietnam’s Transition Experience May be Helpful for North Korea Today ..
time : February 7, 2019 ..
author : Bradley O. Babson ..
content : The recent visit of the DPRK’s Foreign Minister Ri Yong Ho to Hanoi and the announcement that Vietna ..
url : https://www.38north.org/2019/02/bbabson020719/ ..
scrap_time : 2019-02-10-05 05:05:50 ..


title : Misreporting the Trump Administration’s Boffo Break with the Failed North Korea Policy of the Past ..
time : February 5, 2019 ..
author : Leon V. Sigal ..
content : The US Special Representative for North Korea Stephen R. Biegun broke important new ground on admini ..
url : https://www.38north.org/2019/02/lsigal020519/ ..
scrap_time : 2019-02-10-05 05:05:51 ..


title : China’s Sanctions Enforcement and Fuel Prices in North Korea: What the Data Tells Us ..
time : February 1, 2019 ..
author : Benjamin Katzeff Silberstein ..
content : China is by far North Korea’s largest trading partner, accounting for up to 90 percent of its trade  

AttributeError: 'NoneType' object has no attribute 'items'

In [ ]:
from N38_scraper import yield_latest_allnews

begin_date = '2019-01-10'
max_num = 50
sleep = 1.0

for i, json_obj in enumerate(yield_latest_allnews(begin_date, max_num, sleep)):
    title = json_obj['title']
    time = json_obj['time']
    print('[{} / {}] ({}) {}'.format(i+1, max_num, time, title))